# Prompting Techniques

### Zero Shot Prompting
'Zero Shot' means that the prompt will not contain any examples of the desired output. The model will be expected to generate the output from the instruction alone.

In [39]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("../.env")
client = OpenAI()

model = "gpt-4o-mini"



In [40]:
prompt = """
Classify the following text as either positive, neutral or negative sentiment:
Text: I love this product, it's the best!
Sentiment:
"""

def complete(prompt):
    return client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message.content

complete(prompt)

'Sentiment: Positive'

### Few Shot Prompting
'Few Shot' means that the prompt will contain a few examples that can help the model have a more nuanced understanding of the task. Few shot prompting helps in steering model towards more sophisticated outputs. 

In [41]:
prompt = """
Text: I hate this product, it's the worst!
Sentiment: negative
Text: This product is damn good!
Sentiment: positive
Text: This product is okayish
Sentiment: neutral
Text: not sure this I want to buy this product
Sentiment: negative
Text: this product is good, not great
Sentiment: positive
Text: I think this product suits my needs
Sentiment: 
"""
complete(prompt)

'Sentiment: positive'

### CoT (Chain of Thought) Prompting
CoT prompting simulates complex reasoning capabilities by chaining multiple intermediate prompts together. The model is expected to breakdown the task into smaller sub-tasks, explain each step and eventually generate the final output.


In [42]:
prompt = """
Solve the below problem by thinking step by step:
On average Joe throws 25 punches per minute. 
In one fight, he lasts 5 rounds of 3 minutes. 
In another match, he lasts 4 rounds of 5 minutes.
How many punches did he throw in total?
"""
print(complete(prompt))

To find out how many punches Joe threw in total, we need to calculate the number of punches he threw in each fight and then sum them up.

**Step 1: Calculate the total time Joe fought in each match.**

1. **First fight:**
   - Joe lasts 5 rounds of 3 minutes each.
   - Total time = 5 rounds × 3 minutes/round = 15 minutes.

2. **Second fight:**
   - Joe lasts 4 rounds of 5 minutes each.
   - Total time = 4 rounds × 5 minutes/round = 20 minutes.

**Step 2: Calculate the total number of punches thrown in each fight.**

Joe throws an average of 25 punches per minute.

1. **First fight:**
   - Total punches = 25 punches/minute × 15 minutes = 375 punches.

2. **Second fight:**
   - Total punches = 25 punches/minute × 20 minutes = 500 punches.

**Step 3: Calculate the total punches thrown in both fights.**

Total punches = punches from first fight + punches from second fight
Total punches = 375 punches + 500 punches = 875 punches.

**Final Answer:**
Joe threw a total of 875 punches.


### ReAct (Reasoning + Action) Prompting

In [94]:
budget_limits = {
    "food": 500,
    "entertainment": 200,
    "electronics": 1000,
    "clothing": 300,
    "miscellaneous": 150
}

current_spending = {
    "food": 40,
    "entertainment": 10,
    "electronics": 80,
    "clothing": 0,
    "miscellaneous": 0
}

expense_keywords = {
    "food": ["grocery", "restaurant", "food", "meal", "snack"],
    "entertainment": ["movie", "concert", "game", "book", "music"],
    "electronics": ["computer", "phone", "gadget", "laptop", "tablet"],
    "clothing": ["shirt", "pants", "shoes", "jacket", "dress"],
    "miscellaneous": ["gift", "household", "stationery"]
}

def calculate(expression):
    """Performs basic arithmetic operations."""
    try:
        return eval(expression)
    except:
        return "Invalid expression"

def get_expense_category(description):
    """Categorizes an expense based on keywords."""
    description = description.lower()
    for category, keywords in expense_keywords.items():
        if any(keyword in description for keyword in keywords):
            return category
    return "miscellaneous"

def get_budget_limit(category):
    """Returns the budget limit for a given category."""
    return budget_limits.get(category, 0)

def get_current_spending(category):
    """Returns the current spending for a given category."""
    return current_spending.get(category, 0)

def add_expense(amount, description):
    """Adds an expense to the current spending."""
    category = get_expense_category(description)
    current_spending[category] += amount
    return f"Added ${amount} to {category} category."

def get_remaining_budget(category):
    """Calculates the remaining budget for a category."""
    limit = get_budget_limit(category)
    spent = get_current_spending(category)
    return limit - spent

def check_budget_status(category):
    """Checks the budget status for a category."""
    limit = get_budget_limit(category)
    spent = get_current_spending(category)
    remaining = limit - spent
    percentage = (spent / limit) * 100 if limit > 0 else 0
    return f"Category: {category}\nSpent: ${spent}\nBudget: ${limit}\nRemaining: ${remaining}\nPercentage used: {percentage:.2f}%"


In [95]:
tools_desc = {
    "calculate": {
        "description": "Performs basic arithmetic operations (add, subract, multiply, divide) on the provided mathematical expression.",
        "input_params": {
            "expression": "A string containing a mathematical expression, e.g., '2 + 3 * 4'."
        }
    },
    "get_expense_category": {
        "description": "Categorizes an expense based on keywords found in the description. Matches against predefined categories like food, entertainment, electronics, clothing, and miscellaneous.",
        "input_params": {
            "description": "A string describing the expense, e.g., 'Bought a new phone'."
        }
    },
    "get_budget_limit": {
        "description": "Returns the budget limit for a given category based on predefined limits.",
        "input_params": {
            "category": "A string representing the category, e.g., 'food', 'electronics'."
        }
    },
    "get_current_spending": {
        "description": "Returns the current spending amount for a given category.",
        "input_params": {
            "category": "A string representing the category, e.g., 'clothing', 'entertainment'."
        }
    },
    "add_expense": {
        "description": "Adds a specified expense amount to the current spending of the appropriate category, determined based on the description.",
        "input_params": {
            "amount": "A number representing the expense amount, e.g., 50.",
            "description": "A string describing the expense, e.g., 'Dinner at a restaurant'."
        }
    },
    "get_remaining_budget": {
        "description": "Calculates the remaining budget for a specified category by subtracting the current spending from the budget limit.",
        "input_params": {
            "category": "A string representing the category, e.g., 'miscellaneous', 'food'."
        }
    },
    "check_budget_status": {
        "description": "Checks the budget status for a specified category, providing details on the total budget, amount spent, remaining budget, and percentage of budget used.",
        "input_params": {
            "category": "A string representing the category, e.g., 'electronics', 'food'."
        }
    }
}

tools_available = {
    "calculate": calculate,
    "get_expense_category": get_expense_category,
    "get_budget_limit": get_budget_limit,
    "get_current_spending": get_current_spending,
    "add_expense": add_expense,
    "get_remaining_budget": get_remaining_budget,
    "check_budget_status": check_budget_status
}

In [101]:
# ReAct Prompt
PROMPT_TEMPLATE = """
You are an AI assistant capable of reasoning and acting to answer complex questions. Your task is to use the tools provide and the steps already taken to the answer the given question.

# You have access to the following tools:
{tools_desc}

# Steps so far:
{steps}

# To solve the problem, follow these steps:
1. Analyse the question use the ONE tool which could help you the most to answer the question.
2. Observe whether the response the of the tool is enough to answer the question.
3. If not enough, use the another tools that can help you to answer the question.
4. If one tool does not provide the required information, then use try using another tool.
5. Repeat the process until you have enough information to answer the question.
6. Once you have the final answer, respond ONLY with the answer.


Show your reasoning and actions for each step. Use the format:
Thought: your thought process on the question
Action: the tool to be used that can help you to answer the question and its input

Answer the following question:
{question}

{{
    "is_done": true or false, whether the questions has been satisfactorily answered,
    "thought": 'thought process',
    "action": {{'tool_name': '', tool_input: ''}},
    "answer": "use only this key to respond with final answer"
}}
"""

In [102]:
from openai import OpenAI
from dotenv import load_dotenv
import json

load_dotenv("../.env")

class ReActAgent:
    def __init__(self, tools_available, tools_desc: dict, model: str, verbose:bool = True):
        self.tools_available = tools_available
        self.tools_desc = tools_desc
        self.model = model
        self.client = OpenAI()
        self.verbose = verbose

    def complete(self, question, tools_desc, steps) -> dict:
        prompt = PROMPT_TEMPLATE.format(
            tools_desc=tools_desc,
            steps=steps,
            question=question
        )

        response = self.client.chat.completions.create(
            model=self.model,
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": "You are a helpful asssistant who responds in JSON"},
                {"role": "user", "content": prompt}
            ]
        )

        return json.loads(response.choices[0].message.content)
    
    def query(self, question):
        step_count = 0
        steps = []
        final_response = ""

        while True:
            response = self.complete(question, self.tools_desc, steps)
            print(response)
            step_count += 1

            if response["is_done"]:
                final_response = response["answer"]
                break
            
            tool_name, tool_input = response["action"]["tool_name"], response["action"]["tool_input"]
            tool_output = self.tools_available[tool_name](**tool_input)
            step = f"""
            Thought: {response["thought"]}
            Action: {tool_name}({tool_input})
            Observation: {tool_output}
            """
            print(step) if self.verbose else None
            steps.append(step)
        

        return {
            "content": final_response,
            "steps": steps
        }

In [104]:
agent1 = ReActAgent(tools_available, tools_desc, model)

questions = [
    "I just spent $40 on dinner at a restaurant. How much of my food budget is left for the month?",
    "If I buy a $900 laptop and a $200 smartphone, will I exceed my electronics budget?",
    "What percentage of my entertainment budget have I used so far, and how much is left?",
    "I'm planning to buy clothes for $250 and a gift for $100. Can I afford both within my respective budgets?",
    "I've spent $150 on groceries and $50 on a restaurant meal this week. How much more can I spend on food this month?"
    "What categories have I overspent on, and by how much?"
]

question = questions[2]
print(f"Question: {question}")
response = agent1.query(question)
response["content"]

Question: What percentage of my entertainment budget have I used so far, and how much is left?
{'is_done': False, 'thought': 'To find out what percentage of the entertainment budget has been used and how much is left, I first need to check the budget status for the entertainment category. This will provide details on the total budget, amount spent, remaining budget, and the percentage of budget used.', 'action': {'tool_name': 'check_budget_status', 'tool_input': {'category': 'entertainment'}}, 'answer': ''}

            Thought: To find out what percentage of the entertainment budget has been used and how much is left, I first need to check the budget status for the entertainment category. This will provide details on the total budget, amount spent, remaining budget, and the percentage of budget used.
            Action: check_budget_status({'category': 'entertainment'})
            Observation: Category: entertainment
Spent: $10
Budget: $200
Remaining: $190
Percentage used: 5.00%
    

'You have used 5.00% of your entertainment budget, and you have $190 left.'